In [1]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd 

In [14]:
def shuffle_and_group(array):
    np.random.shuffle(array)
    print(array)
    return array[:14],array[14:28],array[28:]

In [16]:
arr = [i for i in range(40)]
a = shuffle_and_group(arr)

[8, 32, 3, 26, 5, 38, 20, 18, 19, 7, 27, 39, 11, 17, 33, 13, 37, 34, 28, 22, 23, 1, 9, 25, 36, 14, 6, 15, 35, 31, 12, 2, 4, 16, 10, 0, 29, 21, 24, 30]


In [17]:
a

([8, 32, 3, 26, 5, 38, 20, 18, 19, 7, 27, 39, 11, 17],
 [33, 13, 37, 34, 28, 22, 23, 1, 9, 25, 36, 14, 6, 15],
 [35, 31, 12, 2, 4, 16, 10, 0, 29, 21, 24, 30])

In [12]:
len(m)

14

In [20]:
a= [0.75, 0.5833333333333334, 0.5833333333333334, 0.5833333333333334, 0.6111111111111112, 0.6, 0.6, 0.6666666666666666, 0.625, 0.5555555555555556, 0.6111111111111112, 0.9166666666666666, 0.7777777777777778, 0.6111111111111112]

In [21]:
len(a)

14

In [44]:
b =  [1,   2,   3,  4,   5,    6,  7,  8,    9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
bstress = [22, 10, 19, 17, 23, 14, 9, 18, 23, 10, 8, 13, 12, 20, 21, 13, 0, 28, 13, 7, 18, 13, 15]
g =  [1,    2,   3,  4,  5,  6,  7,  8,    9, 10, 11, 12, 13, 14, 15, 16, 17,]
gstress =  [11, 13, 25, 7, 28, 14, 8, 13, 10, 12, 11, 16, 20, 25, 16, 31, 9]

In [45]:
for i in range(len(b)):
    bstress[i] = [b[i]-1,bstress[i]]

In [46]:
for i in range(len(g)):
    gstress[i] = [g[i]+22,gstress[i]]    

In [50]:
bstress+=gstress
arrl = []
arrh = []
for i in range(len(bstress)):
    if bstress[i][1]<=13:
        arrl.append(bstress[i][0])
    elif bstress[i][1]>=14:
        arrh.append(bstress[i][0])

In [53]:
arrh

[0, 2, 3, 4, 5, 7, 8, 13, 14, 17, 20, 22, 25, 27, 28, 34, 35, 36, 37, 38]